In [1]:
from google.colab import files
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import applications

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import time

In [2]:
if not os.path.exists(os.path.expanduser('~')+'/.kaggle'):
    ! mkdir ~/.kaggle
os.chdir(os.path.expanduser('~')+'/.kaggle')
if not os.path.exists(os.path.expanduser('~')+'/.kaggle/kaggle.json'):
    kaggle_api_file = files.upload()
    ! kaggle datasets download -d grassknoted/asl-alphabet
    ! unzip -q asl-alphabet.zip
    ! rm -rf asl_alphabet_train/asl_alphabet_train/del

Saving kaggle.json to kaggle.json
 99% 1.01G/1.03G [00:08<00:00, 134MB/s]
100% 1.03G/1.03G [00:08<00:00, 124MB/s]


In [3]:
! ls

asl_alphabet_test  asl_alphabet_train  asl-alphabet.zip  kaggle.json


# Creating Models

In [4]:
def get_data(preprocessing_function):
    image_gen = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   brightness_range=(0.2, 1),
                                   shear_range=45,
                                   zoom_range=(0.5, 1.5),
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   preprocessing_function=preprocessing_function,
                                   validation_split=0.2)

    train_data = 'asl_alphabet_train/asl_alphabet_train'

    train_gen = image_gen.flow_from_directory(train_data,
                                              target_size=(224, 224),
                                              class_mode='categorical',
                                              color_mode='rgb',
                                              shuffle=True,
                                              batch_size=128,
                                              seed=0,
                                              subset='training')

    val_gen = image_gen.flow_from_directory(train_data,
                                            target_size=(224, 224),
                                            class_mode='categorical',
                                            color_mode='rgb',
                                            shuffle=True,
                                            batch_size=128,
                                            seed=0,
                                            subset='validation')

    return train_gen, val_gen

In [5]:
def get_model(model):
    kwargs =    {'include_top':False,
                 'weights':'imagenet',
                 'input_shape':(224, 224, 3),
                 'pooling':'avg'}
    
    base_model = model(**kwargs)
    
    end_model = models.Sequential()
    
    end_model.add(layers.Dense(128))
    end_model.add(layers.ReLU())
    end_model.add(layers.Dense(128))
    end_model.add(layers.ReLU())
    end_model.add(layers.Dense(28, activation='softmax'))

    model = models.Model(inputs=base_model.input, outputs=end_model(base_model.output))

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Training Models

In [7]:
models_dict = {
    'Xception': {'model': applications.xception.Xception,
                 'preprocess_func': applications.xception.preprocess_input},
    'VGG16': {'model': applications.vgg16.VGG16,
              'preprocess_func': applications.vgg16.preprocess_input},
    'VGG19': {'model': applications.vgg19.VGG19,
              'preprocess_func': applications.vgg19.preprocess_input},
    'ResNet50': {'model': applications.resnet.ResNet50,
                 'preprocess_func': applications.resnet.preprocess_input},
    'ResNet50V2': {'model': applications.resnet_v2.ResNet50V2,
                   'preprocess_func': applications.resnet_v2.preprocess_input},
    'ResNet101V2': {'model': applications.resnet_v2.ResNet101V2,
                    'preprocess_func': applications.resnet_v2.preprocess_input},
    'InceptionV3': {'model': applications.inception_v3.InceptionV3,
                    'preprocess_func': applications.inception_v3.preprocess_input},
    'MobileNet': {'model': applications.mobilenet.MobileNet,
                  'preprocess_func': applications.mobilenet.preprocess_input},
    'MobileNetV2': {'model': applications.mobilenet_v2.MobileNetV2,
                    'preprocess_func': applications.mobilenet_v2.preprocess_input},
    'DenseNet201': {'model': applications.densenet.DenseNet201,
                    'preprocess_func': applications.densenet.preprocess_input},
    'EfficientNetB7': {'model': applications.efficientnet.EfficientNetB7,
                       'preprocess_func': applications.efficientnet.preprocess_input}
}

In [8]:
for name, model in models_dict.items():
    print(name)

    train_gen, val_gen = get_data(model['preprocess_func'])
    m = get_model(model['model'])

    start = time.perf_counter()

    history = m.fit(train_gen,
                    epochs=1,
                    validation_data = val_gen)

    duration = time.perf_counter() - start

    models_dict[name]['time'] = duration
    models_dict[name]['val_acc'] = history.history['val_accuracy'][-1]
    
    print()

Xception
Found 67200 images belonging to 28 classes.
Found 16800 images belonging to 28 classes.
525/525 [==============================] - 1465s 3s/step - loss: 1.5988 - accuracy: 0.5237 - val_loss: 1.6485 - val_accuracy: 0.4988

VGG16
Found 67200 images belonging to 28 classes.
Found 16800 images belonging to 28 classes.
525/525 [==============================] - 1530s 3s/step - loss: 1.6133 - accuracy: 0.5179 - val_loss: 1.4846 - val_accuracy: 0.5522

VGG19
Found 67200 images belonging to 28 classes.
Found 16800 images belonging to 28 classes.
525/525 [==============================] - 1553s 3s/step - loss: 1.6236 - accuracy: 0.5121 - val_loss: 1.5558 - val_accuracy: 0.5321

ResNet50
Found 67200 images belonging to 28 classes.
Found 16800 images belonging to 28 classes.
525/525 [==============================] - 1430s 3s/step - loss: 1.3506 - accuracy: 0.5906 - val_loss: 1.3011 - val_accuracy: 0.5829

ResNet50V2
Found 67200 images belonging to 28 classes.
Found 16800 images belongin

# Evaluating and Visualizing results

In [9]:
model_results = []

for name, _ in models_dict.items():
    model_results.append([name,
                          models_dict[name]['val_acc'],
                          models_dict[name]['time']])
    
results = pd.DataFrame(model_results,
                       columns = ['Model', 'Validation Accuracy', 'Training Time (sec.)'])
results = results.sort_values(by='Validation Accuracy', ascending=False).reset_index(drop=True)

results

,Model,Validation Accuracy,Training Time (sec.)
0,MobileNet,0.659107,1345.838785
1,EfficientNetB7,0.633036,2030.432290
2,DenseNet201,0.617143,1537.559222
3,ResNet101V2,0.603095,1591.406151
4,ResNet50V2,0.598571,1407.882862
5,ResNet50,0.582917,1432.433960
6,VGG16,0.552202,1584.762602
7,VGG19,0.532143,1555.261585
8,MobileNetV2,0.519048,1287.885348
9,Xception,0.498750,1467.064103
